In [1]:
# Upload kaggle.json
from google.colab import files
files.upload()  # Upload your kaggle.json

# Set up Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Install required packages
!pip install transformers timm torchvision


Saving kaggle.json to kaggle.json
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Succes

In [3]:
!kaggle datasets download -d liuqzsc/seam-carving-jpeg-image-forgery-dataset
!unzip seam-carving-jpeg-image-forgery-dataset.zip -d forgery_data


Streaming output truncated to the last 5000 lines.
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3553.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3554.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3555.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3556.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3557.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3558.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3559.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/356.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3560.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3561.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kag

In [4]:
from PIL import Image
from glob import glob
from transformers import ViTFeatureExtractor
import torch
from torch.utils.data import Dataset, DataLoader

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

class ForgeryDataset(Dataset):
    def __init__(self, img_paths, labels):
        self.img_paths = img_paths
        self.labels = labels

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        inputs = feature_extractor(images=img, return_tensors="pt")
        return inputs['pixel_values'].squeeze(), torch.tensor(self.labels[idx])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
auth = glob("forgery_data/authentic_images/*.jpg")
forge = glob("forgery_data/forged_images/*.jpg")
paths = auth + forge
labels = [0]*len(auth) + [1]*len(forge)


In [8]:
from transformers import ViTForImageClassification
import torch
import torch.nn as nn

# ✅ Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load model
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

# ✅ Replace classifier for binary classification
model.classifier = nn.Linear(model.classifier.in_features, 2)

# ✅ Move model to device
model.to(device)


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [9]:
from torch import nn

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


In [10]:
from tqdm import tqdm

EPOCHS = 5  # Increase if GPU & time allow

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        outputs = model(pixel_values=images).logits
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {running_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")


NameError: name 'train_loader' is not defined

In [14]:
from glob import glob
from sklearn.model_selection import train_test_split

auth_imgs = glob("forgery_data/authentic_images/*.jpg")
forg_imgs = glob("forgery_data/forged_images/*.jpg")

all_paths = auth_imgs + forg_imgs
all_labels = [0] * len(auth_imgs) + [1] * len(forg_imgs)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    all_paths, all_labels, test_size=0.2, random_state=42
)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [13]:
!ls forgery_data
!unzip seam-carving-jpeg-image-forgery-dataset.zip -d forgery_data


Streaming output truncated to the last 5000 lines.
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3553.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3554.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3555.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3556.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3557.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3558.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3559.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/356.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3560.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q75/untouched/3561.jpg  
  inflating: forgery_data/seamcarving_kaggle/seamcarving_kag

In [15]:
!ls forgery_data

Readme.txt  seamcarving_kaggle


In [16]:
!find forgery_data -type f | head -20


forgery_data/Readme.txt
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Readme.txt
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/5449.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/1669.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/4422.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/2962.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/3581.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/2088.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/7102.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/200.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/6927.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/4274.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/3762.jpg
forgery_data/seamcarving_kaggle/seamcarving_kaggle/Q40/untouched/1792.jpg
forgery_data/seamcarving_ka

In [17]:
from google.colab import files
files.upload()  # upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json


In [21]:
!kaggle datasets download -d labid93/image-forgery-detection
!unzip image-forgery-detection.zip -d forgery_data


Streaming output truncated to the last 5000 lines.
  inflating: forgery_data/Dataset/Original/2349.jpg  
  inflating: forgery_data/Dataset/Original/235.jpg  
  inflating: forgery_data/Dataset/Original/2350.jpg  
  inflating: forgery_data/Dataset/Original/2351.jpg  
  inflating: forgery_data/Dataset/Original/2352.jpg  
  inflating: forgery_data/Dataset/Original/2353.jpg  
  inflating: forgery_data/Dataset/Original/2354.jpg  
  inflating: forgery_data/Dataset/Original/2355.jpg  
  inflating: forgery_data/Dataset/Original/2356.jpg  
  inflating: forgery_data/Dataset/Original/2357.jpg  
  inflating: forgery_data/Dataset/Original/2358.jpg  
  inflating: forgery_data/Dataset/Original/2359.jpg  
  inflating: forgery_data/Dataset/Original/236.jpg  
  inflating: forgery_data/Dataset/Original/2360.jpg  
  inflating: forgery_data/Dataset/Original/2361.jpg  
  inflating: forgery_data/Dataset/Original/2362.jpg  
  inflating: forgery_data/Dataset/Original/2363.jpg  
  inflating: forgery_data/Dataset

In [22]:
!ls forgery_data



Dataset


In [23]:
from glob import glob

auth_imgs = glob("forgery_data/Dataset/original/*.*")  # Authentic images
forg_imgs = glob("forgery_data/Dataset/forged/*.*")    # Forged images

print(f"Authentic: {len(auth_imgs)}, Forged: {len(forg_imgs)}")


Authentic: 0, Forged: 0


In [24]:
!find forgery_data/Dataset -type f | head -20


forgery_data/Dataset/Original/3147.JPG
forgery_data/Dataset/Original/1669.jpg
forgery_data/Dataset/Original/2962.jpg
forgery_data/Dataset/Original/3581.jpg
forgery_data/Dataset/Original/2088.jpg
forgery_data/Dataset/Original/200.jpg
forgery_data/Dataset/Original/4847.JPG
forgery_data/Dataset/Original/1792.jpg
forgery_data/Dataset/Original/2383.jpg
forgery_data/Dataset/Original/536.jpg
forgery_data/Dataset/Original/2029.jpg
forgery_data/Dataset/Original/5059.JPG
forgery_data/Dataset/Original/5233.jpg
forgery_data/Dataset/Original/4304.JPG
forgery_data/Dataset/Original/2388.jpg
forgery_data/Dataset/Original/2048.jpg
forgery_data/Dataset/Original/5890.jpg
forgery_data/Dataset/Original/5044.JPG
forgery_data/Dataset/Original/2546.jpg
forgery_data/Dataset/Original/5562.JPG


In [26]:
import glob

auth_imgs = []
forg_imgs = []

# Collect all supported image types
image_extensions = ["jpg", "jpeg", "png", "bmp", "tif", "tiff", "webp"]

for ext in image_extensions:
    auth_imgs.extend(glob.glob(f"forgery_data/Dataset/Original/*.{ext}", recursive=True))
    forg_imgs.extend(glob.glob(f"forgery_data/Dataset/Forged/*.{ext}", recursive=True))
    # Also handle upper-case variants (e.g., .JPG)
    auth_imgs.extend(glob.glob(f"forgery_data/Dataset/Original/*.{ext.upper()}", recursive=True))
    forg_imgs.extend(glob.glob(f"forgery_data/Dataset/Forged/*.{ext.upper()}", recursive=True))

print(f"✅ Found {len(auth_imgs)} authentic images and {len(forg_imgs)} forged images.")


✅ Found 6500 authentic images and 6500 forged images.


In [27]:
from sklearn.model_selection import train_test_split

train_paths, test_paths, train_labels, test_labels = train_test_split(
    all_paths, all_labels, test_size=0.2, random_state=42
)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [28]:
# Combine paths and assign labels
all_paths = auth_imgs + forg_imgs
all_labels = [0] * len(auth_imgs) + [1] * len(forg_imgs)

print("✅ Total images:", len(all_paths))


✅ Total images: 13000


In [29]:
from sklearn.model_selection import train_test_split

train_paths, test_paths, train_labels, test_labels = train_test_split(
    all_paths, all_labels, test_size=0.2, random_state=42
)


In [30]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from transformers import ViTFeatureExtractor

# Load ViT feature extractor
extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# Custom Dataset class
class ForgeryDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.paths = image_paths
        self.labels = labels

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        inputs = extractor(images=img, return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze()
        label = torch.tensor(self.labels[idx])
        return pixel_values, label

# Create DataLoaders
train_loader = DataLoader(ForgeryDataset(train_paths, train_labels), batch_size=8, shuffle=True)
test_loader = DataLoader(ForgeryDataset(test_paths, test_labels), batch_size=8)


/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [31]:
from transformers import ViTForImageClassification
import torch.nn as nn
import torch

# Choose GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ViT
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

# Replace classifier head for 2 classes (Authentic, Forged)
model.classifier = nn.Linear(model.classifier.in_features, 2)

model.to(device)


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [32]:
from torch import nn
from tqdm import tqdm

# Define loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
EPOCHS = 5

# Training
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        outputs = model(pixel_values=images).logits
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

    print(f"✅ Epoch {epoch+1} complete. Accuracy: {100 * correct / total:.2f}%")


Epoch 1/5: 100%|██████████| 1300/1300 [07:59<00:00,  2.71it/s, acc=92.8, loss=0.0088]


✅ Epoch 1 complete. Accuracy: 92.75%


Epoch 2/5: 100%|██████████| 1300/1300 [07:40<00:00,  2.82it/s, acc=99, loss=0.00359]


✅ Epoch 2 complete. Accuracy: 98.99%


Epoch 3/5: 100%|██████████| 1300/1300 [07:35<00:00,  2.85it/s, acc=99.7, loss=0.0169]


✅ Epoch 3 complete. Accuracy: 99.71%


Epoch 4/5: 100%|██████████| 1300/1300 [07:36<00:00,  2.85it/s, acc=99.9, loss=0.00027]


✅ Epoch 4 complete. Accuracy: 99.93%


Epoch 5/5: 100%|██████████| 1300/1300 [07:39<00:00,  2.83it/s, acc=100, loss=0.0185]

✅ Epoch 5 complete. Accuracy: 99.99%


In [33]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(pixel_values=images).logits
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"✅ Test Accuracy: {100 * correct / total:.2f}%")


✅ Test Accuracy: 95.58%


In [39]:
from google.colab import files
from PIL import Image
import torch

# Upload an image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Preprocess
img = Image.open(image_path).convert("RGB")
inputs = feature_extractor(images=img, return_tensors="pt")
pixel_values = inputs["pixel_values"].to(device)

# Predict
model.eval()
with torch.no_grad():
    logits = model(pixel_values).logits
    pred = torch.argmax(logits, dim=1).item()

print("🕵️ Prediction:", "Forged" if pred == 1 else "Authentic")


Saving WhatsApp Image 2025-07-11 at 20.19.19_a2a4a8d7.jpg to WhatsApp Image 2025-07-11 at 20.19.19_a2a4a8d7.jpg
🕵️ Prediction: Authentic


In [36]:
torch.save(model.state_dict(), "vit_forgery_model.pth")


In [37]:
from google.colab import files
files.download("vit_forgery_model.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
!pip install -q gradio


In [41]:
import gradio as gr
from PIL import Image
import torch

# Define prediction function
def classify_image(img):
    img = img.convert("RGB")
    inputs = feature_extractor(images=img, return_tensors="pt")
    pixel_values = inputs["pixel_values"].to(device)
    with torch.no_grad():
        outputs = model(pixel_values=pixel_values).logits
        pred = torch.argmax(outputs, dim=1).item()
    return "🟢 Authentic" if pred == 0 else "🔴 Forged"

# Launch interface
interface = gr.Interface(fn=classify_image, inputs="image", outputs="text")
interface.launch(share=True)  # 🔗 share=True gives you a public URL!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f0543add1fa2b3f04e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [42]:
!wget -O forged_sample.jpg https://www.scienceabc.com/wp-content/uploads/2021/05/Photoshop-image-manipulation-before-after.jpg


--2025-07-11 14:53:33--  https://www.scienceabc.com/wp-content/uploads/2021/05/Photoshop-image-manipulation-before-after.jpg
Resolving www.scienceabc.com (www.scienceabc.com)... 104.26.8.7, 104.26.9.7, 172.67.68.10, ...
Connecting to www.scienceabc.com (www.scienceabc.com)|104.26.8.7|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-11 14:53:34 ERROR 404: Not Found.



In [44]:
import gradio as gr
from PIL import Image
import torch

# Ensure model is in evaluation mode
model.eval()

# Define prediction function
def classify_image(img):
    try:

        img = Image.fromarray(img).convert("RGB")
 # Ensure it's 3-channel
        inputs = feature_extractor(images=img, return_tensors="pt")
        pixel_values = inputs["pixel_values"].to(device)

        with torch.no_grad():
            outputs = model(pixel_values=pixel_values).logits
            pred = torch.argmax(outputs, dim=1).item()

        return "🟢 Authentic" if pred == 0 else "🔴 Forged"

    except Exception as e:
        return f"❌ Error: {str(e)}"

# Launch Gradio interface
interface = gr.Interface(fn=classify_image, inputs="image", outputs="text")
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff9a7270d679343750.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [45]:
!wget -O forged_splice.jpg https://raw.githubusercontent.com/erikrose/PyMzed/master/tests/images/spliced_face.jpg


--2025-07-11 15:04:11--  https://raw.githubusercontent.com/erikrose/PyMzed/master/tests/images/spliced_face.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-11 15:04:12 ERROR 404: Not Found.



In [46]:
!wget -O forged_face.jpg https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg


--2025-07-11 15:04:32--  https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91814 (90K) [image/jpeg]
Saving to: ‘forged_face.jpg’

forged_face.jpg     100%[===================>]  89.66K  --.-KB/s    in 0.009s  

2025-07-11 15:04:32 (9.31 MB/s) - ‘forged_face.jpg’ saved [91814/91814]



In [47]:
from PIL import Image

!wget -O test.jpg https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg
img = Image.open("test.jpg")
img.show()


--2025-07-11 15:04:51--  https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91814 (90K) [image/jpeg]
Saving to: ‘test.jpg’

test.jpg            100%[===================>]  89.66K  --.-KB/s    in 0.008s  

2025-07-11 15:04:51 (11.6 MB/s) - ‘test.jpg’ saved [91814/91814]



In [48]:
print("Prediction:", classify_image(img))


Prediction: ❌ Error: a bytes-like object is required, not 'JpegImageFile'
